# Kurvanpassning av mätdata
Vi studerar urladdningen av en kondensator genom ett motstånd. Kondensatorn laddas först upp till en spänning $U_0=5\,\text{V}$ och sedan tar vi mätpunkter av spänningen $U(t)$ för olika tider $t$. 

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 9]
import scipy.optimize as opt
import numpy as np
from io import StringIO

Vi sparar mätdatan som kommer från en Arduino i en textfil här i samma katalog där vi har vår Jupyter notebook eller python skript. Denna fil läser vi in här och får värden i två listor `t` och `U`.

In [ ]:
filnamn = "uz_data_100k_100n.txt"
t, U = np.loadtxt(filnamn, unpack=True)

En graf med mätpunkterna på en linjär skala ser ut så här:

In [ ]:
fig, ax = plt.subplots()
ax.plot(t, U, "r+")

Och samma data med en logaritmis y-axel och linjär x-axel ("_halvlogaritmisk plot_") får vi med dessa kommandon: 

In [ ]:
def modellfunktion(t, U_0, tau):
    return U_0 * np.exp(-t / tau)

I kursen Elektromagnetism får ni höra mer om vad som händer vid laddning och urladdning av kondensatorn. Men den kurva som vi ser i våra mätpunkter följer det teoretiska resultatet som ni kommer få fram då.

Vår modellfunktion till fysiken som händer här är en med tiden exponentiellt avtagande spänning $U(t)$ enligt

$\displaystyle U(t) = U_0\cdot e^{-t/\tau}$ 

med parametrarna $U_0$ och $\tau$, där $U_0$ är spänningen vid tiden $t=0$ och $\tau$ anger hur snabbt spänningen avtar.

In [ ]:
fig, ax = plt.subplots()
ax.semilogy(t,U,"r+")

Till kurvanpassningen behöver vi som vanligt våra startvärden `gissning`. 

In [ ]:
gissning = [5,1e-4]

Hur bra passar vår gissning till våra mätpunkter?<br/>
Vi gör en graf där vi jämför kurvan med mätpunkterna. Till detta skapar vi en ny lista med x-värden `xvals` som är tätare än våra mätpunkter.

In [ ]:
xvals = np.arange(min(t),1.2*max(t),max(t)/1200)

fig, ax = plt.subplots()
ax.plot(t, U, "r+")
ax.plot(xvals, modellfunktion(xvals, *gissning), "b-")

Kan kurvanpassningen lyckas? Låt oss testa.

In [ ]:
optimParams, pcov = opt.curve_fit(modellfunktion, t, U, p0=gissning, method="trf", verbose=2)

Återigen så ger sista raden av utskriften information över kurvanpassningens framgång.

In [ ]:
print("\nU_0 = {:.3f} V   tau = {:.5g} s".format(optimParams[0],optimParams[1]))

Och som tidigare så måste vi förvissa oss om att kurvanpassningen verkligen passar bra till våra mätpunkter. Här vill vi veta både hur anpassningen lyckades på en linjär och en logaritmisk skala - en figur med två grafer sida vid sida ger oss svaret.

In [ ]:
xvals = np.arange(min(t),1.2*max(t),max(t)/1200)
yvals = modellfunktion(xvals, *optimParams)

fig, ax = plt.subplots(ncols=2)
fig.suptitle(filnamn, fontsize=30)
ax[0].text( 0.95, 0.75, 
           r"$U(t)={:.2f}\,\rm{{V}} \cdot e^{{-t/{:.3g}\,\rm{{s}}}}$".format(optimParams[0],optimParams[1]), 
           horizontalalignment='right',
           size=20,
           transform=ax[0].transAxes )
ax[0].plot(t, U, "r+", label="data")
ax[0].plot(xvals, yvals, "b-", label="anpassad")
ax[0].legend()
ax[0].set_xlabel(r"tid $t\,\,\,[s]$")
ax[0].set_ylabel(r"spänning över kondensatorn $U_C\,\,\,[V]$")
ax[1].semilogy(t, U, "r+", label="data")
ax[1].semilogy(xvals, yvals, "b-", label="anpassad")
ax[1].legend()
ax[1].set_xlabel(r"tid $t\,\,\,[s]$")
ax[1].set_ylabel(r"spänning över kondensatorn $U_C\,\,\,[V]$");